<a href="https://colab.research.google.com/github/JamilAlMasry/AICare-SymptomDiseaseChatbot/blob/main/AICare_FlaskInterface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flask

In [3]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from flask import Flask, render_template, request, jsonify
import numpy as np
import json
import tensorflow_hub as hub
from tensorflow.keras.utils import custom_object_scope
import keras


model_path = "/content/drive/My Drive/model_USE.h5"





custom_objects={'KerasLayer':hub.KerasLayer}

with keras.saving.custom_object_scope(custom_objects):
    reconstructed_model = keras.models.load_model(model_path)
print("Model Loaded Successfully")

with open('/content/drive/My Drive/label_mapping.json', 'r') as f:
    label_mapping = json.load(f)

# Invert the mapping to create category_reverse_mapping
category_reverse_mapping = {int(k): v for k, v in label_mapping.items()}
print("Label Loaded Successfully")
print(category_reverse_mapping)

def predict_on_sentence(model, sentence, category_reverse_mapping, top_n=1):

    pred_prob = model.predict([sentence])
    pred_label = np.round(pred_prob).astype(int)[0]


    top_indices = np.argsort(pred_prob[0])[::-1][:top_n]


    top_labels_str = [category_reverse_mapping[i] for i in top_indices]
    top_probs_str = [pred_prob[0][i] for i in top_indices]


    predictions = []
    for label, prob in zip(top_labels_str, top_probs_str):
        predictions.append({"disease": label, "probability": prob * 100})

    return predictions
print("Ready to Run the App")

Model Loaded Successfully
Label Loaded Successfully
{0: '(Vertigo) Paroymsal Positional Vertigo', 1: 'Acne', 2: 'Aids', 3: 'Alcoholic Hepatitis', 4: 'Allergy', 5: 'Arthritis', 6: 'Bronchial Asthma', 7: 'Cervical Spondylosis', 8: 'Chicken Pox', 9: 'Chronic Cholestasis', 10: 'Common Cold', 11: 'Dengue', 12: 'Diabetes', 13: 'Diabetes', 14: 'Dimorphic Hemmorhoids(Piles)', 15: 'Dimorphic Hemorrhoids', 16: 'Drug Reaction', 17: 'Fungal Infection', 18: 'Gastroenteritis', 19: 'Gastroesophageal Reflux Disease', 20: 'Gerd', 21: 'Heart Attack', 22: 'Hepatitis A', 23: 'Hepatitis B', 24: 'Hepatitis C', 25: 'Hepatitis D', 26: 'Hepatitis E', 27: 'Hypertension', 28: 'Hypertension', 29: 'Hyperthyroidism', 30: 'Hypoglycemia', 31: 'Hypothyroidism', 32: 'Impetigo', 33: 'Jaundice', 34: 'Malaria', 35: 'Migraine', 36: 'Osteoarthristis', 37: 'Paralysis (Brain Hemorrhage)', 38: 'Peptic Ulcer Disease', 39: 'Peptic Ulcer Disease', 40: 'Pneumonia', 41: 'Psoriasis', 42: 'Tuberculosis', 43: 'Typhoid', 44: 'Urinary T

In [4]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))
#This is used to be able to open flask through google colab. Run this cell, then run the below cell, then open the direct below link.
# Don't forget to change the file path of the model, json file, chats.html :)

https://9ysaq27simu-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [6]:
from flask import Flask, render_template_string, request, jsonify


app = Flask(__name__)

html_file_path = '/content/drive/My Drive/chats.html'
with open(html_file_path, 'r') as file:
    html_content = file.read()

# Define the route for rendering the HTML content
@app.route('/')
def index():
    # Render the HTML content
    return render_template_string(html_content)

# Define the route for handling POST requests
@app.route("/get", methods=["POST"])
def chat():
    # Process the message from the request
    msg = request.form["msg"]
    predictions = predict_on_sentence(reconstructed_model, msg, category_reverse_mapping)
    print(predictions)
    return predictions

# Run the Flask app
if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Mar/2024 19:57:52] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2024 19:57:53] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 1s 607ms/step


INFO:werkzeug:127.0.0.1 - - [10/Mar/2024 19:58:09] "POST /get HTTP/1.1" 200 -


[{'disease': 'Common Cold', 'probability': 58.572572469711304}]
1/1 [==============================] - 0s 39ms/step


INFO:werkzeug:127.0.0.1 - - [10/Mar/2024 19:58:53] "POST /get HTTP/1.1" 200 -


[{'disease': 'Diabetes', 'probability': 43.86264085769653}]
